In [ ]:
from pathlib import Path
import json
import itertools
import pandas as pd

In [ ]:
tdd_anomalycorr = "/Users/christine/Projects/2023_Riverice/data/breakupdata/derived/TDD_anomaly_correlations.csv"
breakup_stats = "/Users/christine/Projects/2023_Riverice/data/breakupdata/derived/breakupdate_mean_std_1983_2022.csv"

In [ ]:
anomalycorrDF = pd.read_csv(tdd_anomalycorr, header=3, index_col=0)
anomalycorrDF

,date,stationname,location,r2,r,p,DateStamp
0,04-01,ALYESKA,Koyukuk River at Bettles,0.000269,0.016411,0.922096,2000-04-01
1,04-01,ANCHORAGE_MERRILL_FIELD,Koyukuk River at Bettles,0.000444,-0.021081,0.920329,2000-04-01
2,04-01,ANCHORAGE_TED_STEVENS_INTL_AP,Koyukuk River at Bettles,0.004498,0.067068,0.676945,2000-04-01
3,04-01,AUKE_BAY,Koyukuk River at Bettles,0.016624,0.128936,0.427824,2000-04-01
4,04-01,BARROW_AP,Koyukuk River at Bettles,0.000000,0.000000,0.000000,2000-04-01
...,...,...,...,...,...,...,...
98600,06-04,SITKA_AP,Gakona River at Gakona,0.144046,-0.379534,0.026840,2000-06-04
98601,06-04,SKAGWAY_AP,Gakona River at Gakona,0.098393,-0.313677,0.118650,2000-06-04
98602,06-04,TALKEETNA_AP,Gakona River at Gakona,0.333423,-0.577428,0.000349,2000-06-04
98603,06-04,TANANA_AP,Gakona River at Gakona,0.040125,-0.200311,0.263692,2000-06-04


In [ ]:
eagleDF = anomalycorrDF[anomalycorrDF.location == 'Yukon River at Eagle']
eagleDF['DateStamp'] = pd.to_datetime(eagleDF['DateStamp'], format='%Y-%m-%d')
eagleDF

/var/folders/39/6h0p929579n8vrp25cjkbz3h0000gn/T/ipykernel_63839/2464246974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eagleDF['DateStamp'] = pd.to_datetime(eagleDF['DateStamp'], format='%Y-%m-%d')


,date,stationname,location,r2,r,p,DateStamp
1147,04-01,ALYESKA,Yukon River at Eagle,0.060588,-0.246146,1.309109e-01,2000-04-01
1148,04-01,ANCHORAGE_MERRILL_FIELD,Yukon River at Eagle,0.094858,-0.307990,1.341859e-01,2000-04-01
1149,04-01,ANCHORAGE_TED_STEVENS_INTL_AP,Yukon River at Eagle,0.022928,-0.151418,3.324146e-01,2000-04-01
1150,04-01,AUKE_BAY,Yukon River at Eagle,0.013091,-0.114415,4.706087e-01,2000-04-01
1151,04-01,BARROW_AP,Yukon River at Eagle,0.000000,0.000000,0.000000e+00,2000-04-01
...,...,...,...,...,...,...,...
98267,06-04,SITKA_AP,Yukon River at Eagle,0.311553,-0.558170,1.005521e-04,2000-06-04
98268,06-04,SKAGWAY_AP,Yukon River at Eagle,0.305020,-0.552287,8.608987e-04,2000-06-04
98269,06-04,TALKEETNA_AP,Yukon River at Eagle,0.440213,-0.663486,1.240178e-06,2000-06-04
98270,06-04,TANANA_AP,Yukon River at Eagle,0.525176,-0.724690,5.751470e-08,2000-06-04


In [ ]:
grouped = eagleDF.drop(
        columns=['date', 'location', 'r', 'p']).groupby(
                ['stationname', pd.Grouper(key='DateStamp', freq='15D')]).mean().reset_index()

In [ ]:
max4 = (
    grouped
        .groupby(['DateStamp'])
        .apply(lambda group: group.nlargest(4, columns='r2'))
).reset_index(drop=True)

In [ ]:
datestamps = ['2000-04-01', '2000-04-16',
       '2000-05-01', '2000-05-16']

[(['FAIRBANKS_INTL_AP', 'COLLEGE_OBSERVATORY', 'TALKEETNA_AP', 'EAGLE'],),
 (['FAIRBANKS_INTL_AP', 'EAGLE', 'COLLEGE_OBSERVATORY', 'BIG_DELTA_AP'],),
 (['EAGLE', 'FAIRBANKS_INTL_AP', 'COLLEGE_OBSERVATORY', 'BIG_DELTA_AP'],),
 (['FAIRBANKS_INTL_AP', 'COLLEGE_OBSERVATORY', 'EAGLE', 'NENANA_MUN_AP'],)]

In [ ]:
sites = set()
maxN = 10
for datestamp in datestamps:
    for station in max4[max4.DateStamp== datestamp].stationname.to_list():
        if station == 'COLLEGE_OBSERVATORY':
            continue
        if station not in sites:
            break
    print(station)
    sites.add(station)
    if len(sites) == maxN:
        break 

FAIRBANKS_INTL_AP
EAGLE
BIG_DELTA_AP
NENANA_MUN_AP


In [ ]:
sites

{'COLLEGE_OBSERVATORY', 'EAGLE', 'FAIRBANKS_INTL_AP', 'NENANA_MUN_AP'}

In [ ]:
def restore_stationnames(stationlst):
    return [
        item.replace('INTL', 'INTERNATIONAL').replace('AP', 'AIRPORT')
        for item in stationlst
        ]

sites1 = ['COLLEGE_OBSERVATORY', 'EAGLE', 'FAIRBANKS_INTL_AP', 'NENANA_MUN_AP']
sites1 = restore_stationnames(sites1)
sites2 = ['FAIRBANKS_INTL_AP', 'EAGLE', 'BIG_DELTA_AP', 'NENANA_MUN_AP']
sites2 = restore_stationnames(sites2)
sites2

['FAIRBANKS_INTERNATIONAL_AIRPORT',
 'EAGLE',
 'BIG_DELTA_AIRPORT',
 'NENANA_MUN_AIRPORT']